In [1]:
import requests
import warnings, sys
import urllib
import itertools
from requests.exceptions import ConnectionError, HTTPError, Timeout, TooManyRedirects
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import traceback
import logging
import logging.handlers
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy as sa
from sqlalchemy import create_engine, event
import json
import time as ti
import datetime
from datetime import datetime
import re
import os
import asyncio
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import ctypes
import threading

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

global stage, onb, conn, df, recnum #, GCVList
conn = False
con = None

#GCVList = ['GCV_PRD','GCV_STG','GCV_STG_ONB']
try:
    
    logging.basicConfig(filename = 'CommonConfigLog.log',
                        filemode='a',
                        format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                        datefmt='%H:%M:%S',
                        level=logging.DEBUG)
    logging.info(f'\nLOG START: {datetime.now()}\n')
    
    #def open_connection():
    #    global server, database, driver, dformat, connection, con
    #    with open(r'C:\Users\gmoye001\config\config.json', 'r') as fh:
    #        config = json.load(fh)
    #    server = config['server']
    #    database = config['database']
    #    driver = config['driver']
    #    dformat = config['dformat']
    #    connection = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes'
    #    con = pyodbc.connect(connection)
    #    return con
    #
    #def connection_test():
    #    cur = con.cursor()
    #    cur.execute("SELECT @@version")
    #    row = cur.fetchone()
    #    cur.close()
    #    con.commit()
    #    logging.info(f"\nConnection established to: {row[0]}\n")
    #    return print("Connection established to: ",row[0])
    #    
    #def close_connection():
    #    con.close()
    #    return
    
    def open_connection():
        global server, database, driver, dformat, connection, con
        with open(r"config\config.json", 'r') as fh:
            config = json.load(fh)
        server = config['server']
        database = config['database']
        driver = config['driver']
        dformat = config['dformat']
        connection = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes'
        con = pyodbc.connect(connection)
        return con
    
    def connection_test():
        cur = con.cursor()
        cur.execute("SELECT @@version")
        row = cur.fetchone()
        print("Connection established to: ",row[0])
        cur.close()
        con.commit()
        logging.info(f"\nConnection established to: {row[0]}\n")
        return 
        
    def close_connection():
        con.close()
        conn = False
        return
    
    def checktables(con, tbl):
        if (stage == False and onb == False):
            cur = con.cursor()
            query = "SELECT * FROM information_schema.Tables WHERE table_schema = 'GCV_PRD' and table_name = '%s'" % tbl
            cur.execute(query)
            output = cur.fetchall()
            if output == []:
                output = f'\n[GCV_PRD].{tbl} does not exist in the database and will need to be created\n'
                print(output, end = "\r")
                logging.info(output, end = "\r")
                cur.close()
                return False
            else:
                print(output, end = "\r")
                cur.close()
            return True
        elif (stage == True and onb == False):
            cur = con.cursor()
            query = "SELECT * FROM information_schema.Tables WHERE table_schema = 'GCV_STG' and table_name = '%s'" % tbl
            cur.execute(query)
            output = cur.fetchall()
            if output == []:
                output = f'\n[GCV_STG].{tbl} does not exist in the database and will need to be created\n'
                print(output, end = "\r")
                logging.info(output, end = "\r")
                cur.close()
                return False
            else:
                print(output, end = "\r")
                cur.close()
            return True
        elif (stage == False and onb == True):
            cur = con.cursor()
            query = "SELECT * FROM information_schema.Tables WHERE table_schema = 'GCV_STG_ONB' and table_name = '%s'" % tbl
            cur.execute(query)
            output = cur.fetchall()
            if output == []:
                output = f'\n[GCV_STG_ONB].{tbl} does not exist in the database and will need to be created\n'
                print(output, end = "\r")
                logging.info(output, end = "\r")
                cur.close()
                return False
            else:
                print(output, end = "\r")
                cur.close()
            return True

    def checkbackups(con, tbl):
        if (stage == False and onb == False):
            cur = con.cursor()
            query = "SELECT * FROM information_schema.Tables WHERE table_schema = 'GCV_PRD' and table_name = '%s_backup'" % tbl
            cur.execute(query)
            output = cur.fetchall()
            if output == []:
                output = f'\n[GCV_PRD].{tbl}_backup does not exist in the database and will need to be created once parent table has data\n'
                print(output, end = "\r")
                logging.info(output)
                cur.close()
                return False
            else:
                print(f'\nCurrent backup for [GCV_PRD].{tbl} exists.', end = "\r")
                logging.info(f'\nCurrent backup for [GCV_PRD].{tbl} exists.\n')
                cur.close()
                return True
        elif (stage == True and onb == False):
            cur = con.cursor()
            query = "SELECT * FROM information_schema.Tables WHERE table_schema = 'GCV_STG' and table_name = '%s_backup'" % tbl
            cur.execute(query)
            output = cur.fetchall()
            if output == []:
                output = f'\n[GCV_STG].{tbl}_backup does not exist in the database and will need to be created once parent table has data\n'
                print(output, end = "\r")
                logging.info(output)
                cur.close()
                return False
            else:
                print(f'\nCurrent backup for [GCV_STG].{tbl}', end = "\r")
                logging.info(f'\nCurrent backup for [GCV_STG].{tbl}\n')
                cur.close()
                return True
        elif (stage == False and onb == True):
            cur = con.cursor()
            query = "SELECT * FROM information_schema.Tables WHERE table_schema = 'GCV_STG_ONB' and table_name = '%s_backup'" % tbl
            cur.execute(query)
            output = cur.fetchall()
            if output == []:
                output = f'\n[GCV_STG_ONB].{tbl}_backup does not exist in the database and will need to be created once parent table has data\n'
                print(output, end = "\r")
                logging.info(output)
                cur.close()
                return False
            else:
                print(f'\nCurrent backup for [GCV_STG_ONB].{tbl}', end = "\r")
                logging.info(f'\nCurrent backup for [GCV_STG_ONB].{tbl}\n')
                cur.close()
                return True
    
    def tablecontent(con, tbl):
        if (stage == False and onb == False):
            cur = con.cursor()
            query = "SELECT COUNT(*) FROM [GCV_PRD].[%s]" % tbl
            cur.execute(query)
            output = cur.fetchone()
            #print(output)
            if output[0] == 0:
                print(f'\n{output[0]} Existing records', end = "\r")
                logging.info(f'\n{output[0]} Existing records\n')
                cur.close()
                return False
            else:
                print(f'\n{output[0]} Existing records', end = "\r")
                logging.info(f'\n{output[0]} Existing records\n')
                cur.close()
                return True
        elif (stage == True and onb == False):
            cur = con.cursor()
            query = "SELECT COUNT(*) FROM [GCV_STG].[%s]" % tbl
            cur.execute(query)
            output = cur.fetchone()
            #print(output)
            if output[0] == 0:
                print(f'\n{output[0]} Existing records', end = "\r")
                logging.info(f'\n{output[0]} Existing records\n')
                cur.close()
                return False
            else:
                print(f'\n{output[0]} Existing records', end = "\r")
                logging.info(f'\n{output[0]} Existing records\n')
                cur.close()
                return True
        elif (stage == False and onb == True):
            cur = con.cursor()
            query = "SELECT COUNT(*) FROM [GCV_STG_ONB].[%s]" % tbl
            cur.execute(query)
            output = cur.fetchone()
            #print(output)
            if output[0] == 0:
                print(f'\n{output[0]} Existing records', end = "\r")
                logging.info(f'\n{output[0]} Existing records\n')
                cur.close()
                return False
            else:
                print(f'\n{output[0]} Existing records', end = "\r")
                logging.info(f'\n{output[0]} Existing records\n')
                cur.close()
                return True
        
    def backup_data(con, tbl):
        if (stage == False and onb == False):
            if is_identity(con, tbl) == True:
                try:
                    bstart_time = datetime.now()
                    cur = con.cursor()
                    query = f"SET IDENTITY_INSERT [GCV_PRD].[%s_backup] ON; INSERT INTO [GCV_PRD].[%s_backup] ({fields}) SELECT {fields} FROM [GCV_PRD].[%s]; SET IDENTITY_INSERT [GCV_PRD].[%s_backup] OFF;" % (tbl, tbl, tbl, tbl)
                    #print(f'\n{query}\n')
                    cur.execute(query)
                    con.commit()
                    cur.close()
                    bend_time = datetime.now()
                    print('\nBackup Duration: {}'.format(bend_time - bstart_time))
                    duration = bend_time - bstart_time
                    logging.info(f'\nBackup Duration: {duration}')
                    print("")
                    logging.info('\n')   
                except(Exception, pyodbc.DatabaseError) as e:
                    print("")
                    logging.info('\n')
                    print(e)
                    logging.exception("message")
                    cur.close()
                    con.rollback()
            else:
                try:
                    bstart_time = datetime.now()
                    cur = con.cursor()
                    query = "INSERT INTO [GCV_PRD].[%s_backup] SELECT * FROM [GCV_PRD].[%s]" % (tbl, tbl)
                    cur.execute(query)
                    con.commit()
                    cur.close()
                    bend_time = datetime.now()
                    print('\nBackup Duration: {}'.format(bend_time - bstart_time))
                    duration = bend_time - bstart_time
                    logging.info(f'\nBackup Duration: {duration}')
                    print("")
                    logging.info('\n')   
                except(Exception, pyodbc.DatabaseError) as e:
                    print("")
                    logging.info('\n')
                    print(e)
                    logging.exception("message")
                    cur.close()
                    con.rollback()
            return
        elif (stage == True and onb == False):
            if is_identity(con, tbl) == True:
                try:
                    bstart_time = datetime.now()
                    cur = con.cursor()
                    query = f"SET IDENTITY_INSERT [GCV_STG].[%s_backup] ON; INSERT INTO [GCV_STG].[%s_backup] ({fields}) SELECT {fields} FROM [GCV_STG].[%s]; SET IDENTITY_INSERT [GCV_STG].[%s_backup] OFF;" % (tbl, tbl, tbl, tbl)
                    #print(f'\n{query}\n')
                    cur.execute(query)
                    con.commit()
                    cur.close()
                    bend_time = datetime.now()
                    print('\nBackup Duration: {}'.format(bend_time - bstart_time))
                    duration = bend_time - bstart_time
                    logging.info(f'\nBackup Duration: {duration}')
                    print("")
                    logging.info('\n')   
                except(Exception, pyodbc.DatabaseError) as e:
                    print("")
                    logging.info('\n')
                    print(e)
                    logging.exception("message")
                    cur.close()
                    con.rollback()
            else:
                try:
                    bstart_time = datetime.now()
                    cur = con.cursor()
                    query = "INSERT INTO [GCV_STG].[%s_backup] SELECT * FROM [GCV_STG].[%s]" % (tbl, tbl)
                    cur.execute(query)
                    con.commit()
                    cur.close()
                    bend_time = datetime.now()
                    print('\nBackup Duration: {}'.format(bend_time - bstart_time))
                    duration = bend_time - bstart_time
                    logging.info(f'\nBackup Duration: {duration}')
                    print("")
                    logging.info('\n')   
                except(Exception, pyodbc.DatabaseError) as e:
                    print("")
                    logging.info('\n')
                    print(e)
                    logging.exception("message")
                    cur.close()
                    con.rollback()
            return
        elif (stage == False and onb == True):
            if is_identity(con, tbl) == True:
                try:
                    bstart_time = datetime.now()
                    cur = con.cursor()
                    query = f"SET IDENTITY_INSERT [GCV_STG_ONB].[%s_backup] ON; INSERT INTO [GCV_STG_ONB].[%s_backup] ({fields}) SELECT {fields} FROM [GCV_STG].[%s]; SET IDENTITY_INSERT [GCV_STG].[%s_backup] OFF;" % (tbl, tbl, tbl, tbl)
                    #print(f'\n{query}\n')
                    cur.execute(query)
                    con.commit()
                    cur.close()
                    bend_time = datetime.now()
                    print('\nBackup Duration: {}'.format(bend_time - bstart_time))
                    duration = bend_time - bstart_time
                    logging.info(f'\nBackup Duration: {duration}')
                    print("")
                    logging.info('\n')   
                except(Exception, pyodbc.DatabaseError) as e:
                    print("")
                    logging.info('\n')
                    print(e)
                    logging.exception("message")
                    cur.close()
                    con.rollback()
            else:
                try:
                    bstart_time = datetime.now()
                    cur = con.cursor()
                    query = "INSERT INTO [GCV_STG_ONB].[%s_backup] SELECT * FROM [GCV_STG_ONB].[%s]" % (tbl, tbl)
                    cur.execute(query)
                    con.commit()
                    cur.close()
                    bend_time = datetime.now()
                    print('\nBackup Duration: {}'.format(bend_time - bstart_time))
                    duration = bend_time - bstart_time
                    logging.info(f'\nBackup Duration: {duration}')
                    print("")
                    logging.info('\n')   
                except(Exception, pyodbc.DatabaseError) as e:
                    print("")
                    logging.info('\n')
                    print(e)
                    logging.exception("message")
                    cur.close()
                    con.rollback()
            return

    def is_identity(con, tbl):
        if (stage == False and onb == False):
            try:
                cur = con.cursor()
                query = f"SELECT is_identity FROM sys.identity_columns WHERE object_id in ( SELECT id FROM sysobjects WHERE NAME = '%s')" % (tbl)
                #print(f'\n{query}\n')
                cur.execute(query)
                output = cur.fetchone()
                if output is None:
                    #print(f'\n{output}\n')
                    print(f'\nID Check: Table does not contain a PK_ID\n')
                    return False
                else:
                    output = output[0]
                    #print(f'\n{output}\n')
                    print(f'\nID Check: Table contains a PK_ID, executing special backup procedures.\n')
                    return True
                con.commit()
                cur.close()
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
            return 
        elif (stage == True and onb == False):
            try:
                cur = con.cursor()
                query = "SELECT is_identity FROM sys.identity_columns WHERE object_id in ( SELECT id FROM sysobjects WHERE NAME = '%s')" % (tbl)
                #print(f'\n{query}\n')
                cur.execute(query)
                output = cur.fetchone()
                if output is None:
                    #print(f'\n{output}\n')
                    print(f'\nID Check: Table does not contain a PK_ID\n')
                    return False
                else:
                    output = output[0]
                    #print(f'\n{output}\n')
                    print(f'\nID Check: Table contains a PK_ID, executing special backup procedures.\n')
                    return True
                con.commit()
                cur.close()
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
            return
        elif (stage == False and onb == True):
            try:
                cur = con.cursor()
                query = "SELECT is_identity FROM sys.identity_columns WHERE object_id in ( SELECT id FROM sysobjects WHERE NAME = '%s')" % (tbl)
                #print(f'\n{query}\n')
                cur.execute(query)
                output = cur.fetchone()
                if output is None:
                    #print(f'\n{output}\n')
                    print(f'\nID Check: Table does not contain a PK_ID\n')
                    return False
                else:
                    output = output[0]
                    #print(f'\n{output}\n')
                    print(f'\nID Check: Table contains a PK_ID, executing special backup procedures.\n')
                    return True
                con.commit()
                cur.close()
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
            return
            
    def get_fields(con, tbl):
        global fields
        if (stage == False and onb == False):
            schema = 'GCV_PRD'
            try:
                bstart_time = datetime.now()
                cur = con.cursor()
                query = f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = '%s' AND TABLE_NAME = '%s'" % (schema, tbl)
                #print(f'\n{query}\n')
                cur.execute(query)
                columns = cur.fetchall()
                columns = columns[0:]
                columns = list(zip(*columns))[0]
                fields = ", ".join(map(str, columns))
                #print(f'\n{fields}\n')
                con.commit()
                cur.close()
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
            return fields
        elif (stage == True and onb == False):
            schema = 'GCV_STG'
            try:
                bstart_time = datetime.now()
                cur = con.cursor()
                query = f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = '%s' AND TABLE_NAME = '%s'" % (schema, tbl)
                #print(f'\n{query}\n')
                cur.execute(query)
                columns = cur.fetchall()
                columns = columns[0:]
                columns = list(zip(*columns))[0]
                fields = ", ".join(map(str, columns))
                #print(f'\n{fields}\n')
                con.commit()
                cur.close()
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
            return fields
        elif (stage == False and onb == True):
            schema = 'GCV_STG_ONB'
            try:
                bstart_time = datetime.now()
                cur = con.cursor()
                query = f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = '%s' AND TABLE_NAME = '%s'" % (schema, tbl)
                #print(f'\n{query}\n')
                cur.execute(query)
                columns = cur.fetchall()
                columns = columns[0:]
                columns = list(zip(*columns))[0]
                fields = ", ".join(map(str, columns))
                #print(f'\n{fields}\n')
                con.commit()
                cur.close()
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
            return fields
            

 
    def create_backuptable(con, tbl):
        if (stage == False and onb == False):
            try:
                bstart_time = datetime.now()
                cur = con.cursor()
                query = "SELECT * INTO [GCV_PRD].[%s_backup] FROM [GCV_PRD].[%s]" % (tbl, tbl)
                cur.execute(query)
                con.commit()
                cur.close()
                bend_time = datetime.now()
                logging.info(f'\n[GCV_PRD].{tbl} backup table has been created.\n')
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
                return False
            return True
        elif (stage == True and onb == False):
            try:
                bstart_time = datetime.now()
                cur = con.cursor()
                query = "SELECT * INTO [GCV_STG].[%s_backup] FROM [GCV_STG].[%s]" % (tbl, tbl)
                cur.execute(query)
                con.commit()
                cur.close()
                bend_time = datetime.now()
                logging.info(f'\n[GCV_STG].{tbl} backup table has been created.\n')
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
                return False
            return True
        elif (stage == False and onb == True):
            try:
                bstart_time = datetime.now()
                cur = con.cursor()
                query = "SELECT * INTO [GCV_STG_ONB].[%s_backup] FROM [GCV_STG_ONB].[%s]" % (tbl, tbl)
                cur.execute(query)
                con.commit()
                cur.close()
                bend_time = datetime.now()
                logging.info(f'\n[GCV_STG_ONB].{tbl} backup table has been created.\n')
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
                return False
            return True
        
    def backupcheck(con, tbl):
        global rowcount
        if (stage == False and onb == False):
            cur = con.cursor()
            query = "SELECT COUNT(*) FROM [GCV_PRD].[%s] UNION ALL SELECT COUNT(*) FROM [GCV_PRD].[%s_backup]" % (tbl, tbl)
            cur.execute(query)
            output = cur.fetchone()
            rowcount = []
            while output is not None:
                rowcount.append(output[0])
                output = cur.fetchone()
            print('\n# of records in each table: ', rowcount[0], recnum, end = "\r")
            logging.info(f'\n# of records in each table: {rowcount[0]}, {recnum}')
            cur.close()
            if recnum == rowcount[0]:
                truncate_backup(con, tbl)
                get_fields(con, tbl)
                backup_data(con, tbl)
            elif recnum < rowcount[0]:
                truncate_backup(con, tbl)
                get_fields(con, tbl)
                backup_data(con, tbl)
            elif rowcount[0] == 0:
                pass
            return rowcount
        elif (stage == True and onb == False):
            cur = con.cursor()
            query = "SELECT COUNT(*) FROM [GCV_STG].[%s] UNION ALL SELECT COUNT(*) FROM [GCV_STG].[%s_backup]" % (tbl, tbl)
            cur.execute(query)
            output = cur.fetchone()
            rowcount = []
            while output is not None:
                rowcount.append(output[0])
                output = cur.fetchone()
            print('\n# of records in each table: ', rowcount[0], recnum, end = "\r")
            logging.info(f'\n# of records in each table: {rowcount[0]}, {recnum}')
            cur.close()
            if recnum == rowcount[0]:
                truncate_backup(con, tbl)
                get_fields(con, tbl)
                backup_data(con, tbl)
            elif recnum < rowcount[0]:
                truncate_backup(con, tbl)
                get_fields(con, tbl)
                backup_data(con, tbl)
            elif rowcount[0] == 0:
                pass
            return rowcount
        elif (stage == False and onb == True):
            cur = con.cursor()
            query = "SELECT COUNT(*) FROM [GCV_STG_ONB].[%s] UNION ALL SELECT COUNT(*) FROM [GCV_STG_ONB].[%s_backup]" % (tbl, tbl)
            cur.execute(query)
            output = cur.fetchone()
            rowcount = []
            while output is not None:
                rowcount.append(output[0])
                output = cur.fetchone()
            print('\n# of records in each table: ', rowcount[0], recnum, end = "\r")
            logging.info(f'\n# of records in each table: {rowcount[0]}, {recnum}')
            cur.close()
            if recnum == rowcount[0]:
                truncate_backup(con, tbl)
                get_fields(con, tbl)
                backup_data(con, tbl)
            elif recnum < rowcount[0]:
                truncate_backup(con, tbl)
                get_fields(con, tbl)
                backup_data(con, tbl)
            elif rowcount[0] == 0:
                pass
            return rowcount
    
    def truncate_table(con, tbl):
        if (stage == False and onb == False):
            try:
                cur = con.cursor()
                query = "TRUNCATE TABLE [GCV_PRD].[%s]" % tbl
                cur.execute(query)
                print(f'\n[GCV_PRD].{tbl} has been succesfully truncated to import new data.')
                logging.info(f'\n[GCV_PRD].{tbl} has been succesfully truncated to import new data.\n')
                cur.close()
                con.commit()
            except Exception as err:
                logging.exception("message")
                cur.close()
                con.rollback()
                #raise err
        elif (stage == True and onb == False):
            try:
                cur = con.cursor()
                query = "TRUNCATE TABLE [GCV_STG].[%s]" % tbl
                cur.execute(query)
                print(f'\n[GCV_STG].{tbl} has been succesfully truncated to import new data.')
                logging.info(f'\n[GCV_STG].{tbl} has been succesfully truncated to import new data.\n')
                cur.close()
                con.commit()
            except Exception as err:
                logging.exception("message")
                cur.close()
                con.rollback()
                #raise err
        elif (stage == False and onb == True):
            try:
                cur = con.cursor()
                query = "TRUNCATE TABLE [GCV_STG_ONB].[%s]" % tbl
                cur.execute(query)
                print(f'\n[GCV_STG_ONB].{tbl} has been succesfully truncated to import new data.')
                logging.info(f'\n[GCV_STG_ONB].{tbl} has been succesfully truncated to import new data.\n')
                cur.close()
                con.commit()
            except Exception as err:
                logging.exception("message")
                cur.close()
                con.rollback()
                #raise err
            
    def truncate_backup(con, tbl):
        if (stage == False and onb == False):
            try:
                cur = con.cursor()
                query = "TRUNCATE TABLE [GCV_PRD].[%s_backup]" % tbl
                cur.execute(query)
                print(f'\n[GCV_PRD].{tbl}_backup has been succesfully truncated to create the next backup.')
                logging.info(f'\n[GCV_PRD].{tbl}_backup has been succesfully truncated to create the next backup.\n')
                cur.close()
                con.commit()
            except Exception as err:
                logging.exception("message")
                cur.close()
                con.rollback()
                #raise err
        elif (stage == True and onb == False):
            try:
                cur = con.cursor()
                query = "TRUNCATE TABLE [GCV_STG].[%s_backup]" % tbl
                cur.execute(query)
                print(f'\n[GCV_STG].{tbl}_backup has been succesfully truncated to create the next backup.')
                logging.info(f'\n[GCV_STG].{tbl}_backup has been succesfully truncated to create the next backup.\n')
                cur.close()
                con.commit()
            except Exception as err:
                logging.exception("message")
                cur.close()
                con.rollback()
                #raise err
        elif (stage == False and onb == True):
            try:
                cur = con.cursor()
                query = "TRUNCATE TABLE [GCV_STG_ONB].[%s_backup]" % tbl
                cur.execute(query)
                print(f'\n[GCV_STG_ONB].{tbl}_backup has been succesfully truncated to create the next backup.')
                logging.info(f'\n[GCV_STG_ONB].{tbl}_backup has been succesfully truncated to create the next backup.\n')
                cur.close()
                con.commit()
            except Exception as err:
                logging.exception("message")
                cur.close()
                con.rollback()
                #raise err

    def importdata(con, tbl):
        if (stage == False and onb == False):
            cur = con.cursor()
            engine = sa.create_engine(f'mssql+pyodbc://{server}/{database}?driver={dformat}', fast_executemany = True)
            #pd.io.sql._is_sqlalchemy_connectable(engine)
            df.to_sql(f'{tbl}', engine, index = False, if_exists = 'append', schema = 'GCV_PRD')
            cur.close()
            con.commit()
            return
        elif (stage == True and onb == False):
            cur = con.cursor()
            engine = sa.create_engine(f'mssql+pyodbc://{server}/{database}?driver={dformat}', fast_executemany = True)
            #pd.io.sql._is_sqlalchemy_connectable(engine)
            df.to_sql(f'{tbl}', engine, index = False, if_exists = 'append', schema = 'GCV_STG')
            cur.close()
            con.commit()
            return
        elif (stage == False and onb == True):
            cur = con.cursor()
            engine = sa.create_engine(f'mssql+pyodbc://{server}/{database}?driver={dformat}', fast_executemany = True)
            #pd.io.sql._is_sqlalchemy_connectable(engine)
            df.to_sql(f'{tbl}', engine, index = False, if_exists = 'append', schema = 'GCV_STG_ONB')
            cur.close()
            con.commit()
            return
            
    def getData(CV):
        # convert to config file/table
        if (stage == False and onb == False):
            url = f'https://api.pwcinternal.com:7443/GlobalCVService/GlobalCVService.svc/cv/{CV}'
            with open(r'config\apiconnect.json') as f:
                headers = json.load(f)
        elif (stage == True and onb == False):
            url = f'https://api-staging.pwcinternal.com:7443/GlobalCVService/GlobalCVService.svc/cv/{CV}'
            with open(r'config\apiconnectb.json') as f:
                headers = json.load(f)
        elif (stage == False and onb == True):
            url = f'https://api-staging.pwcinternal.com:7443/GlobalCVService/GlobalCVService.svc/cv/{CV}'
            with open(r'config\apiconnectb.json') as f:
                headers = json.load(f)
                
        retry_strategy = Retry(total = 10, status_forcelist=[429, 413, 503], method_whitelist=["HEAD", "GET", "PUT", "DELETE", "OPTIONS", "TRACE"])
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        
        http.mount("https://", adapter)
        r = http.get(url, headers=headers, timeout = 100)
        rjson = r.json()
        keylist = ('URI','Categories','RelatedTerms')
        
        if 'ErrorMessage' in rjson:
            if rjson['ErrorMessage'] == 'CV does not exist in CVMaster List':
                print('CV Currently Missing from CVMaster List')
                return
        else:
            for key in keylist:
                rjson = [{k: v for k, v in d.items() if k != key} for d in rjson]
        
            global df, recnum
            df = pd.DataFrame(rjson)
            datelist = ('CreatedDate','ModifiedDate','EffectiveDate', 'HierarchyNodeEffectiveDate', 'HierarchyNodeExpirationDate','RelModifiedDate','ExpiryDate','Effective_Date','Expiration_Date','Created_Datetime','Last_Modified_Datetime')
            date_format = "%Y%m%d%H%M%S"
            
            for date in enumerate(datelist): 
                if date[1] in df:
                    
                    if df[date[1]].iteritems() != 'None':
                        if df[date[1]].str.contains('Z').items():
                            df[date[1]] = df[date[1]].str.replace("\.[0-9]*Z", "").str.replace("Z", "")
                    if df[date[1]].iteritems() != 'None':   
                        if df[date[1]].str.contains('-').items():
                            df[date[1]] = df[date[1]].str.replace("\W+", "")
                            
                    df[date[1]] = df[date[1]].mask(df[date[1]].str.len() > 14, df[date[1]].str[:-3])
                    df[date[1]] = pd.to_datetime(df[date[1]], format=date_format, errors = 'coerce')
                    
                else:
                    continue
                
            recnum = len(df.index)
            
            #display(df)
            #df.to_excel('output1.xlsx')
            
            if tablecontent(con, f'{CV}') == False:
                importdata(con, f'{CV}')
                print(f'\n{CV} has been updated with {recnum} records')
                logging.info(f'\n{CV} has been updated with {recnum} records\n')
            else:
                truncate_table(con, f'{CV}')
                importdata(con, f'{CV}')
                print(f'\n{CV} has been updated with {recnum} records')
                logging.info(f'\n{CV} has been updated with {recnum} records\n')
                
            #colnames = list(df)
            
            #display(colnames) 
             #[['Effective_Date','Expiration_Date','Created_Datetime','Last_Modified_Datetime']])
            
            #--fetch column names for table creation and datatypes
            #for colname, dt in itertools.product([df.columns],[df.dtypes]):
                #print(dt)
            #print('\n')
        
            return df
    
    
    
    with open(r'config\CVsStage.csv', 'r') as cv_config:
        CVsStage = cv_config.read().split(',')
    with open(r'config\CVsProd.csv', 'r') as cv_config:
        CVsProd = cv_config.read().split(',')
    with open(r'config\CVsONB.csv', 'r') as cv_config:
        CVsONB = cv_config.read().split(',')

    #Used for testing a group of CV's
    CVa = ['LEL-PwCLegalEntity-en','NS-PwCNetworkNode-en',
           'NS-PwCNetworkNode-en-Territory',
           'ORD-CostCenter']
    #Used for testing a single CV
    CVx = ['ORD-CostCenter']
    
    CVd = ['ORD-CostCenter']
    
    st = datetime.now()
    open_connection()
    connection_test()
    close_connection()
    
    for urls in enumerate(CVsStage):
        global stage
        stage = True
        onb = False
        conn = True
        open_connection()
        #getData(urls[1])
        #close_connection()
        if checktables(con, urls[1]) == False:
            print("Moving to next table.\n", end = "\r")
        else:
            if checkbackups(con, urls[1]) == False:
                create_backuptable(con, urls[1])
                createdbackup = True
            else:
                createdbackup = False
            try:
                gstart_time = datetime.now()
                getData(urls[1])
                if createdbackup == False:
                    backupcheck(con, urls[1])
                else:
                    pass
                close_connection()
                stage = False
                conn = False
                gend_time = datetime.now()
                print('\nDuration: {}'.format(gend_time - gstart_time))
                logging.info('\nDuration: {}'.format(gend_time - gstart_time))
                print("")
            except (Exception, pyodbc.DatabaseError) as e:
                print("")
                print(e)
                logging.exception('\n')
                logging.exception("message")
        for i in range(10, -1, -1):
                print(f"{i} seconds until next table is imported ", end = "\r")
                ti.sleep(1)

    for urls in enumerate(CVsProd):
        open_connection()
        stage = False
        onb = False
        conn = True
        #getData(urls[1])
        #close_connection()
        if checktables(con, urls[1]) == False:
            print("Moving to next table.\n", end = "\r")
        else:
            if checkbackups(con, urls[1]) == False:
                create_backuptable(con, urls[1])
                createdbackup = True
            else:
                createdbackup = False
            try:
                gstart_time = datetime.now()
                getData(urls[1])
                if createdbackup == False:
                    backupcheck(con, urls[1])
                else:
                    pass
                close_connection()
                conn = False
                gend_time = datetime.now()
                print('\nDuration: {}'.format(gend_time - gstart_time))
                logging.info('\nDuration: {}'.format(gend_time - gstart_time))
                print("")
            except (Exception, pyodbc.DatabaseError) as e:
                print("")
                print(e)
                logging.exception('\n')
                logging.exception("message")
        for i in range(10, -1, -1):
                print(f"{i} seconds until next table is imported ", end = "\r")
                ti.sleep(1)

    for urls in enumerate(CVsONB):
        open_connection()
        stage = False
        onb = True
        conn = True
        #getData(urls[1])
        #close_connection()
        if checktables(con, urls[1]) == False:
            print("Moving to next table.\n", end = "\r")
        else:
            if checkbackups(con, urls[1]) == False:
                create_backuptable(con, urls[1])
                createdbackup = True
            else:
                createdbackup = False
            try:
                gstart_time = datetime.now()
                getData(urls[1])
                if createdbackup == False:
                    backupcheck(con, urls[1])
                else:
                    pass
                close_connection()
                onb = False
                conn = False
                gend_time = datetime.now()
                print('\nDuration: {}'.format(gend_time - gstart_time))
                logging.info('\nDuration: {}'.format(gend_time - gstart_time))
                print("")
            except (Exception, pyodbc.DatabaseError) as e:
                print("")
                print(e)
                logging.exception('\n')
                logging.exception("message")
        for i in range(10, -1, -1):
                print(f"{i} seconds until next table is imported ", end = "\r")
                ti.sleep(1)
            
    et = datetime.now()
    print('Total Execution Duration: {}'.format(et - st),'\n-Import Completed-')
    tt = et - st
    logging.info(f'\nTotal Execution Duration: {tt}\n')
    logging.info('\n-Import Completed-\n')
    
#exceptions 
except (Exception, pyodbc.DatabaseError) as error:
        print(error)
        logging.exception("message")
        pass
    
except requests.exceptions.HTTPError as errh:
    print("Http Error:",  errh)
    conn = False
    logging.exception("message")
    
except requests.exceptions.ConnectionError as errc:
    print("Error Connecting:", errc)
    conn = False
    logging.exception("message")
    
except requests.exceptions.Timeout as errt:
    print("Timeout Error:", errt)
    conn = False
    logging.exception("message")
    
except requests.exceptions.RequestException as erru:
    print("Unidentified Request Exception:", erru)
    conn = False
    logging.exception("message")
        
finally:
    logging.info(f'\nLOG END: {datetime.now()}\n')
    if conn == True:
        close_connection()

Connection established to:  Microsoft SQL Server 2019 (RTM-CU18) (KB5017593) - 15.0.4261.1 (X64) 
	Sep 12 2022 15:07:06 
	Copyright (C) 2019 Microsoft Corporation
	Enterprise Edition: Core-based Licensing (64-bit) on Windows Server 2016 Datacenter 10.0 <X64> (Build 14393: ) (Hypervisor)

[('GCV_API', 'GCV_STG', 'NS-ManagementLevel-en', 'BASE TABLE')]


<ipython-input-1-54cfce87e31b>:839: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  if df[date[1]].iteritems() != 'None':
<ipython-input-1-54cfce87e31b>:841: FutureWarning: The default value of regex will change from True to False in a future version.
  df[date[1]] = df[date[1]].str.replace("\.[0-9]*Z", "").str.replace("Z", "")
<ipython-input-1-54cfce87e31b>:842: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  if df[date[1]].iteritems() != 'None':
<ipython-input-1-54cfce87e31b>:844: FutureWarning: The default value of regex will change from True to False in a future version.
  df[date[1]] = df[date[1]].str.replace("\W+", "")
<ipython-input-1-54cfce87e31b>:839: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  if df[date[1]].iteritems() != 'None':
<ipython-input-1-54cfce87e31b>:841: FutureWarning: The default value of regex will 


0 Existing records
NS-ManagementLevel-en has been updated with 12 records

# of records in each table:  12 12
[GCV_STG].NS-ManagementLevel-en_backup has been succesfully truncated to create the next backup.


ID Check: Table does not contain a PK_ID


Backup Duration: 0:00:00.231129


Duration: 0:00:07.796450

Total Execution Duration: 0:00:21.400224 
-Import Completed-
